Часть 1

In [40]:
# Импорт библиотек
import numpy as np
from keras.preprocessing import image
import os
import cv2
from natsort import natsorted
from skimage.feature import hog
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import re

In [41]:
# Загрузка изображений
def load_image(folder_path, target_size=(45, 45)):
  imgs = []
  lables = []
  lable = 0
  
  dirs = natsorted(list(os.walk(folder_path))[0][1])
  named_labels = dirs
  for dir_ in dirs:
    curr_dir = folder_path + '/' + dir_
    os.chdir(curr_dir)
    files = natsorted(list(os.walk(curr_dir))[0][2])
    file_cnt = 0
    for file in files:
      img = cv2.imread(file)
      img_resized = cv2.resize(img, target_size)

      img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
      img_norm = img_gray / 255.0

      clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
      img_enhanced = clahe.apply((img_norm * 255).astype(np.uint8))
      
      imgs.append(hog(img_enhanced, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), block_norm="L2-Hys", visualize=False))
      
      lables.append(lable)
      if file_cnt == 99:
        lable += 1
        break
      file_cnt += 1

  return imgs, lables, named_labels


In [42]:
# Получение изображений, меток и символов? соответствующих меткам классов
imgs, lables, named_labels = load_image("/home/RuDe/Space/Institute/МФТИ/Python/Dev/Semester_10/hw_3/data")

In [43]:
# Создание конвейера для стандиртизации данных
pipe = Pipeline([("scaler", StandardScaler()), ("knn", KNeighborsClassifier())])
pipe.fit(imgs, lables)

Pipeline(steps=[('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])

In [44]:
# Получение тестового математического выражения для распознавания
test_path = "/home/RuDe/Space/Institute/МФТИ/Python/Dev/Semester_10/hw_3/test_exp"
files = natsorted(list(os.walk(test_path))[0][2])
target_size = (45, 45)

test_imgs = []

os.chdir(test_path)
for file in files:
  img = cv2.imread(file)
  img_resized = cv2.resize(img, target_size)

  img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
  img_norm = img_gray / 255.0

  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  img_enhanced = clahe.apply((img_norm * 255).astype(np.uint8))
  
  test_imgs.append(hog(img_enhanced, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), block_norm="L2-Hys", visualize=False))

In [45]:
# Функция для вывода результата распознавания
def show_res(y_pred):
  for y in y_pred:
    print(named_labels[y], end='')

In [46]:
# Распознавание математического выражения
y_pred = pipe.predict(test_imgs)
show_res(y_pred)

(1-9)

In [47]:
# Разделение данных на выборки
X_train, X_test, y_train, y_test = train_test_split(imgs, lables, test_size=0.3, random_state=42)

In [48]:
# Перебор гиперпараметров модели KNN
best = {"weight": '', "metric": '', "neighbor_num": 0, "acc": -1}
for weight in ["uniform", "distance"]:
  for metric in ["minkowski", "cosine", "euclidean"]:
    for neighbor_num in range(2, 60):
      pipe = Pipeline([("scaler", StandardScaler()), ("knn", KNeighborsClassifier(n_neighbors=neighbor_num, weights=weight, metric=metric))])
      pipe.fit(X_train, y_train)
      acc = accuracy_score(y_test, pipe.predict(X_test))
      if best["acc"] < acc:
        best["weight"] = weight
        best["metric"] = metric
        best["neighbor_num"] = neighbor_num
        best["acc"] = acc

print(best)

{'weight': 'distance', 'metric': 'cosine', 'neighbor_num': 9, 'acc': 0.8095238095238095}


In [49]:
# Обучение модели KNN
pipe = Pipeline([("scaler", StandardScaler()), ("knn", KNeighborsClassifier(n_neighbors=best["neighbor_num"], weights=best["weight"], metric=best["metric"]))])
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('knn',
                 KNeighborsClassifier(metric='cosine', n_neighbors=9,
                                      weights='distance'))])

In [50]:
# Распознавание тестового математического выражения
y_pred = pipe.predict(test_imgs)
show_res(y_pred)

(1-9)

In [51]:
# Функция для получения строки с распознанным математическим выражением по меткам классов
def get_expression(y_pred):
  ret = ""

  for y in y_pred:
    ret += named_labels[y]

  return ret

In [52]:
# Получение строки с математическим выражением
expr = get_expression(y_pred)
print(expr)

(1-9)


In [53]:
# Проверка корректности результата распознавания
checked_expr = re.fullmatch(r"\(\d+[-+*/^]\d+\)", expr)
if checked_expr is not None:
  print("Correct!")
  print(eval(expr))
else:
  print("Incorrect!")

Correct!
-8


Часть 2

In [54]:
# Получение тестового математического выражения
test_path = "/home/RuDe/Space/Institute/МФТИ/Python/Dev/Semester_10/hw_3/test_exp"
files = natsorted(list(os.walk(test_path))[0][2])
target_size = (45, 45)

test2_imgs = []

os.chdir(test_path)
for file in files:
  img = cv2.imread(file)
  img_resized = cv2.resize(img, target_size)

  img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
  img_norm = img_gray / 255.0

  test2_imgs.append(img_norm)

In [56]:
# Соединение изображений симоволов
concat_img = cv2.hconcat(test2_imgs) 

In [58]:
# Кодирование изображения 0 и 1
mask_size = (5, 5)
res_str = ""
for row in range(0, concat_img.shape[0], 5):
  for col in range(0, concat_img.shape[0], 5):
    average_value = sum(sum(concat_img[row:row + 5, col:col + 5])) / (mask_size[0] * mask_size[1])
    if average_value < 1:
      res_str += '0'
    else:
      res_str += '1'

In [59]:
# Поиск шаблона в закодированной строке изображения
print("Найдено совпадений:", len(re.findall(r"010", res_str)))

Найдено совпадений: 0
